# Tutorial: Train PyTorch Models

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
features, labels = datasets.load_breast_cancer(return_X_y=True)
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2)

In [3]:
features.shape, labels.shape

((569, 30), (569,))

In [4]:
import torch
import torch.nn as nn
import torch.utils.data as D
from kuma_utils.torch import TorchTrainer, EarlyStopping, TorchLogger
from kuma_utils.torch.model_zoo import TabularNet
from kuma_utils.metrics import AUC, Accuracy
from kuma_utils.utils import sigmoid

torch_xla not found.
nvidia apex not found.


In [5]:
train_ds = D.TensorDataset(
    torch.as_tensor(x_train).float(),
    torch.as_tensor(y_train.reshape(-1, 1)).float()
)
valid_ds = D.TensorDataset(
    torch.as_tensor(x_valid).float(),
    torch.as_tensor(y_valid.reshape(-1, 1)).float()
)
train_loader = D.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_loader = D.DataLoader(valid_ds, batch_size=64, shuffle=False)

In [9]:
model = TabularNet(in_features=x_train.shape[1], out_features=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
LogitsAcc = lambda approx, target: AUC().torch(sigmoid(approx), target)

fit_params = {
    'loader': train_loader,
    'loader_valid': valid_loader,
    'criterion': criterion,
    'optimizer': optimizer,
    'scheduler': scheduler,
    'num_epochs': 30,
    'callbacks': [EarlyStopping(5, maximize=True, skip_epoch=10)],
    'eval_metric': AUC().torch,
    'monitor_metrics': [LogitsAcc], 
    'logger': TorchLogger('logger.log', log_items=[
        'loss_train', 'loss_valid', 'metric_valid', 'monitor_metrics_valid', 'earlystop'
    ]),
    # 'calibrate_model': True, 
    # 'snapshot_path': 'results/baseline/',
    # 'resume': True
}

TorchLogger created at 20/11/21:03:21:21


In [10]:
trn = TorchTrainer(model)
trn.train(**fit_params)

03:21:22 Model is on cpu
03:21:22 loss_train=4.437667 | loss_valid=0.769077 | metric_valid=0.035100 | monitor_metrics_valid=[0.035100] 
03:21:22 loss_train=1.761077 | loss_valid=0.635398 | metric_valid=0.861004 | monitor_metrics_valid=[0.861004] 
03:21:22 loss_train=1.513724 | loss_valid=0.704916 | metric_valid=0.058968 | monitor_metrics_valid=[0.058968] 
03:21:22 loss_train=1.055435 | loss_valid=0.610955 | metric_valid=0.936820 | monitor_metrics_valid=[0.936820] 
03:21:22 loss_train=0.864968 | loss_valid=0.625825 | metric_valid=0.963847 | monitor_metrics_valid=[0.963847] 
03:21:22 loss_train=0.744723 | loss_valid=0.634138 | metric_valid=0.958231 | monitor_metrics_valid=[0.958231] 
03:21:22 loss_train=0.681240 | loss_valid=0.623364 | metric_valid=0.975079 | monitor_metrics_valid=[0.975079] 
Epoch     8: reducing learning rate of group 0 to 5.0000e-04.
03:21:22 loss_train=0.647059 | loss_valid=0.601147 | metric_valid=0.966655 | monitor_metrics_valid=[0.966655] 
03:21:22 loss_train=0.654